<a href="https://colab.research.google.com/github/Uma29-10/TNSIF_AIML_DBIT/blob/main/reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


ROWS, COLS = 4, 4
TERMINAL = (3, 3)

ACTIONS = {
    'U': (-1, 0),
    'D': (1, 0),
    'L': (0, -1),
    'R': (0, 1)
}

GAMMA = 0.9
THETA = 1e-4
STEP_REWARD = -1
TERMINAL_REWARD = 0

# All states as (r, c) tuples
states = [(r, c) for r in range(ROWS) for c in range(COLS)]

def is_terminal(s):
    return s == TERMINAL

def next_state(s, action):
    """Deterministic transition with wall bounce (stay if would go off-grid)."""
    if is_terminal(s):
        return s
    dr, dc = ACTIONS[action]
    nr = max(0, min(ROWS - 1, s[0] + dr))
    nc = max(0, min(COLS - 1, s[1] + dc))
    return (nr, nc)

def reward(s, s2):
    """Reward for moving from s to s2."""
    return TERMINAL_REWARD if is_terminal(s2) else STEP_REWARD

# Initialize values to zero
V = {s: 0.0 for s in states}

# Value Iteration
iteration = 0
while True:
    iteration += 1
    delta = 0.0
    newV = V.copy()  # synchronous update

    for s in states:
        if is_terminal(s):
            continue
        # compute value for each action (deterministic => single next state)
        action_values = []
        for a in ACTIONS:
            s2 = next_state(s, a)
            r = reward(s, s2)
            q = r + GAMMA * V[s2]
            action_values.append(q)

        best = max(action_values)
        newV[s] = best
        delta = max(delta, abs(V[s] - newV[s]))

    V = newV
    if delta < THETA:
        break

# Extract greedy policy
policy = {}
arrow = {'U': '^', 'D': 'v', 'L': '<', 'R': '>'}

for s in states:
    if is_terminal(s):
        policy[s] = 'T'
        continue
    best_a = None
    best_q = float('-inf')
    for a in ACTIONS:
        s2 = next_state(s, a)
        q = reward(s, s2) + GAMMA * V[s2]
        if q > best_q:
            best_q = q
            best_a = a
    policy[s] = arrow[best_a]

# Print values and policy in grid form
print("Converged in", iteration, "iterations\n")
print("Value function (rows top->bottom):")
for r in range(ROWS):
    for c in range(COLS):
        print(f"{V[(r,c)]:6.2f}", end=" ")
    print()

print("\nPolicy (T=terminal):")
for r in range(ROWS):
    for c in range(COLS):
        print(f" {policy[(r,c)]}", end=" ")
    print()

Converged in 6 iterations

Value function (rows top->bottom):
 -4.10  -3.44  -2.71  -1.90 
 -3.44  -2.71  -1.90  -1.00 
 -2.71  -1.90  -1.00   0.00 
 -1.90  -1.00   0.00   0.00 

Policy (T=terminal):
 v  v  v  v 
 v  v  v  v 
 v  v  v  v 
 >  >  >  T 
